In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
pip install transformers requests beautifulsoup4 pandas numpy

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

Initialize model

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Passing tokens to model and test

In [44]:
tokens = tokenizer.encode('I feel amazing.', return_tensors='pt')

In [45]:
result = model(tokens)

In [46]:
# get result on a scale of 1-5 : 5 being positive
int(torch.argmax(result.logits))+1

5

Collect reviews

In [73]:
#Scraping the reviews : modify the parameters based on your project
r=requests.get('https://www.yelp.com/biz/ozzys-burgers-vaughan-vaughan')
soup= BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results=soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

Load reviews into data frame and score

In [67]:
import numpy as np
import pandas as pd

In [68]:
#Create a dataframe and pass the reviews scraped above
df = pd.DataFrame(np.array(reviews), columns=['reviews'])

In [69]:
def sentiment_score(review):
  tokens=tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [70]:
sentiment_score(df['reviews'].iloc[1])

4

In [71]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df